<a href="https://colab.research.google.com/github/karsarobert/RAG/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Understanding and Implementing Retrieval Augmented Generation for Enhanced LLMs


API keys:

https://drive.google.com/file/d/1h-6bgnpQ-TG1Ul2bN8-Lm3Tolyj5qtPG/view?usp=sharing

##Wordtovec

In [ ]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 452.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 9.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.3
    Uninstalling scipy-1.15.3:
      Successfully uninstalled scipy-1.15.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.
thinc 8.3.6 req

Restart the runtime

In [ ]:
import gensim.downloader as api
from pprint import pprint


print("Loading pre-trained Word2Vec model...")
model = api.load("glove-wiki-gigaword-100")
print("Model loaded.")


word = "bank"
print(f"\n '{word}' vector (first 20 elements):")
print(model[word][:20])


print(f"\n Words most similar to '{word}':")
pprint(model.most_similar(word, topn=5))


pairs = [("king", "queen"), ("king", "apple"), ("man", "woman")]
print("\n Word similarity scores:")
for w1, w2 in pairs:
    similarity = model.similarity(w1, w2)
    print(f"{w1} - {w2}: {similarity:.4f}")


Loading pre-trained Word2Vec model...
[==================================================] 100.0% 128.1/128.1MB downloaded
Model loaded.

🔹 'bank' vector (first 20 elements):
[ 0.41869   -0.92211    0.048684   0.11798    0.22062   -0.32506
 -0.78456    0.43787   -0.14762    0.0631     0.23754    1.4837
 -0.54762    0.80406   -0.0037728  0.24949   -0.15523   -0.66263
 -0.060428   0.10121  ]

🔹 Words most similar to 'bank':
[('banks', 0.8057132959365845),
 ('banking', 0.7530707120895386),
 ('credit', 0.7037603259086609),
 ('investment', 0.6939943432807922),
 ('financial', 0.6777413487434387)]

🔹 Word similarity scores:
king - queen: 0.7508
king - apple: 0.2668
man - woman: 0.8323


### queen = (king - man) + woman

In [ ]:
model.most_similar(positive=['king', 'woman'], negative=['man'], topn=5)

[('queen', 0.7698540687561035),
 ('monarch', 0.6843381524085999),
 ('throne', 0.6755736470222473),
 ('daughter', 0.6594556570053101),
 ('princess', 0.6520534157752991)]

##BERT Bidirectional Encoder Representations from Transformers

In [ ]:
import gensim
from gensim.models import Word2Vec
from transformers import BertTokenizerFast, BertModel
import torch
import numpy as np

# Sample text with different contexts
sentences = [
    ["I", "went", "to", "the", "bank", "to", "deposit", "money"],
    ["The", "river", "bank", "was", "covered", "with", "trees"],
    ["I", "love", "machine", "learning"],
    ["Machine", "learning", "is", "great"],
    ["I", "love", "transformers"]
]

# Training Word2Vec model
w2v_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Word2Vec vector (same in all contexts)
word = "bank"
contexts = [
    "I went to the bank to deposit money",
    "The river bank was covered with trees"
]
w2v_vector = w2v_model.wv[word]
print("\nWord2Vec vector:")
print(f"{word}: {w2v_vector[:5]}...")

# Load BERT
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

bert_model = BertModel.from_pretrained('bert-base-uncased')

def get_bert_contextual_embeddings(sentence, target_word, tokenizer, model):
    # Tokenize with offsets to match the original word
    inputs = tokenizer(sentence, return_tensors='pt', return_offsets_mapping=True, add_special_tokens=True)
    with torch.no_grad():
        outputs = model(**{k: v for k, v in inputs.items() if k != 'offset_mapping'})
    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
    offsets = inputs['offset_mapping'][0]

    # Find token indices that correspond to the word "bank"
    word_positions = []
    for i, (token, offset) in enumerate(zip(tokens, offsets)):
        if offset[0] == 0 and offset[1] == 0:
            continue  # special tokens like [CLS], [SEP]
        token_text = sentence[offset[0]:offset[1]]
        if token_text.lower() == target_word:
            word_positions.append(i)

    if not word_positions:
        raise ValueError(f"'{target_word}' not found in sentence: {sentence}")

    # Get the hidden states for those positions and average them
    embeddings = outputs.last_hidden_state[0]
    word_embedding = torch.stack([embeddings[pos] for pos in word_positions], dim=0).mean(dim=0)
    return word_embedding.numpy()

# Get BERT vectors in different contexts
bert_vectors = [get_bert_contextual_embeddings(context, word, tokenizer, bert_model) for context in contexts]

print("\nBERT vectors in different contexts:")
for i, vector in enumerate(bert_vectors):
    print(f"Context {i+1}: {contexts[i]}")
    print(f"{word}: {vector[:5]}...")

# Cosine similarity
def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

print("\nCosine similarities in BERT contexts:")
for i in range(len(bert_vectors)):
    for j in range(i+1, len(bert_vectors)):
        b1, b2 = bert_vectors[i], bert_vectors[j]
        bert_sim = cosine_similarity(b1, b2)
        print(f"Context {i+1} - Context {j+1}: BERT = {bert_sim:.4f}")

# Word2Vec similarity
print("\nWord2Vec similarity:")
print(f"{word} - {word}: Word2Vec = 1.0000")



Word2Vec vector:
bank: [-0.00713656  0.00123667 -0.00718363 -0.00224445  0.00371492]...


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]


BERT vectors in different contexts:
Context 1: I went to the bank to deposit money
bank: [ 0.48843178 -0.24248171  0.05825159 -0.15357924  1.0020232 ]...
Context 2: The river bank was covered with trees
bank: [-0.19330327 -0.37276787 -0.12327462 -0.2277071  -0.36730555]...

Cosine similarities in BERT contexts:
Context 1 - Context 2: BERT = 0.5035

Word2Vec similarity:
bank - bank: Word2Vec = 1.0000


In [ ]:
!pip -q install langchain openai tiktoken PyPDF2 faiss-cpu langchain_community langchain-huggingface langchain-openai sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 91.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.9/62.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56

# RAG

In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

## The Game plan


<img src="https://dl.dropboxusercontent.com/s/gxij5593tyzrvsg/Screenshot%202023-04-26%20at%203.06.50%20PM.png" alt="vectorstore">


<img src="https://dl.dropboxusercontent.com/s/v1yfuem0i60bd88/Screenshot%202023-04-26%20at%203.52.12%20PM.png" alt="retreiver chain">


In [ ]:
# Download the PDF Reid Hoffman book with GPT-4 from his free download link
!wget -q https://www.impromptubook.com/wp-content/uploads/2023/03/impromptu-rh.pdf


### Basic Chat PDF


In [ ]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

## Reading in the PDF


In [ ]:
# location of the pdf file/files.
doc_reader = PdfReader('/content/impromptu-rh.pdf')


In [ ]:
doc_reader

In [ ]:
# read data from the file and put them into a variable called raw_text
raw_text = ''
for i, page in enumerate(doc_reader.pages):
    text = page.extract_text()
    if text:
        raw_text += text

In [ ]:
len(raw_text)

371090

In [ ]:
raw_text[:500]

'Impromptu\nAmplifying Our Humanity \nThrough AI\nBy Reid Hoffman  \nwith GPT-4Impromptu: AmplIfyIng our HumAnIty tHrougH AI  \nby Reid Hoffman with GPT-4\nISBNs:  979-8-9878319-1-5 Trade Paperback  \n979-8-9878319-2-2 Hardcover \n979-8-9878319-0-8 Ebook\nCopyright 2023 Dallepedia LLC\nPublished by Dallepedia LLC. All rights reserved. No portion \nof this work may be reproduced in any form, with the limited \nexception of brief quotations in editorial reviews, without express \nwritten permission from the pub'

### Text Splitter

This takes the text and splits it into chunks. The chunk size is characters not tokens

In [ ]:
# Splitting up the text into smaller chunks for indexing
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 1000,
    chunk_overlap  = 200, #striding over the text
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [ ]:
len(texts)

466

In [ ]:
texts[20]

'Because, really, an AI book? When things are moving so \nquickly? Even with a helpful AI on hand to speed the process, \nany such book would be obsolete before we started to write it—\nthat’s how fast the industry is moving.\nSo I hemmed and hawed for a bit. And then I thought of a frame \nthat pushed me into action.\nThis didn’t have to be a comprehensive “book” book so much as \na travelog, an informal exercise in exploration and discovery, \nme (with GPT-4) choosing one path among many. A snapshot \nmemorializing—in a subjective and decidedly not definitive \nway—the AI future we were about to experience.\nWhat would we see? What would impress us most? What would \nwe learn about ourselves in the process? Well aware of the brief \nhalf-life of this travelog’s relevance, I decided to press ahead.\nA month later, at the end of November 2022, OpenAI released \nChatGPT, a “conversational agent,” aka chatbot, a modified \nversion of GPT-3.5 that they had fine-tuned through a process'

In [ ]:
texts[10]

'one. They’re just going to stare at you with that smug \nlook on their face and say, “You know this is a serious \nsafety hazard. You could have a fire, you could have a \nlawsuit, you could have a bad Yelp review. You better get \non that, pal.” And then they leave, and you’re left with a \ndark corner of your dining room, where no one wants to \nsit, where the food looks unappetizing, where the mood \nis gloomy. And you think to yourself, how many restau-\nrant inspectors does it take to change a lightbulb? None. \nThey don’t change lightbulbs. They just make you feel \nbad about yours.\nPretty good, right? Here’s another:\nReid: How many restaurant inspectors does it take to \nchange a lightbulb? Answer in the style of the philoso -\npher Ludwig Wittgenstein. \nGPT-4:  The question of how many restaurant inspectors \nit takes to change a lightbulb cannot be answered with \na simple number. In fact, the answer to this question lies \nmore in the realm of language than it does in the

## Making the embeddings

In [ ]:
# Download embeddings from OpenAI
# embeddings = OpenAIEmbeddings()

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

# Making the embeddings using HuggingFaceEmbeddings
# You can choose different models from the HuggingFace library
# Here we use 'sentence-transformers/all-MiniLM-L6-v2' as an example
hf_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
#hf_embeddings = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-small")
# You can now use 'hf_embeddings' instead of 'embeddings' for creating the vectorstore


In [ ]:
docsearch = FAISS.from_texts(texts, hf_embeddings)

In [ ]:
docsearch.embedding_function

HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [ ]:
query = "how does GPT-4 change social media?"
docs = docsearch.similarity_search(query)

In [ ]:
len(docs)

4

In [ ]:
docs[0]

Document(id='f0d77779-4f79-49ea-b55a-f71bc17741d9', metadata={}, page_content='be spread across multiple pages. Getting your answer will still \nchew up a lot of your time.\nIn addition, imagine when your goal is to get a concise overview \nof some broader interest or theme, like those I suggested at \nthe top of this chapter (researching supply chain management \nissues during COVID-19, or a travel itinerary that reflects spe -\ncific interests).\nThe takeaway: in your overall quest for authoritative infor-\nmation, GPT-4 helps you start somewhere much closer to the \nfinish line than if you didn’t have it as a resource. \nMore importantly, it possesses this capability because it is able \nto access and synthesize the web’s information in a significantly \ndifferent way from existing information resources like Wikipe-\ndia or traditional search engines. Essentially, GPT-4 arranges \nvast, unstructured arrays of human knowledge and expres -\nsion into a more connected and interoperable

## Plain QA Chain

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain_openai import OpenAI

<img src="https://miro.medium.com/v2/resize:fit:720/format:webp/1*nn1mc2rG9sGuqH16abWGGw.png">

The stuff method is really nice because it’s pretty simple. We just put all of it into one prompt and send that to the language model and get back one response.

##OpenAi API

In [ ]:
chain = load_qa_chain(OpenAI(),
                      chain_type="stuff") # we are going to stuff all the docs in at once

<ipython-input-22-912069a1b081>:1: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/docs/how_to/#qa-with-rag
  chain = load_qa_chain(OpenAI(),


In [ ]:
# check the prompt
print(chain.llm_chain.prompt.template)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:

In [ ]:
query = "who are the authors of the book?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

<ipython-input-24-c2e3bd2dc88c>:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(input_documents=docs, question=query)


' Donald Johanson and di Cesare. Ishiguro also mentions his own novels, but they are not explicitly stated to be part of the book. '

##Huggingface Endpoint

In [ ]:
from langchain_huggingface import HuggingFaceEndpoint
from google.colab import userdata
HF_TOKEN= userdata.get('HF_TOKEN')


llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-3.1-8B-Instruct",
    #repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1",
    huggingfacehub_api_token=HF_TOKEN,
    task="text-generation",
)

chain = load_qa_chain(llm,
                      chain_type="stuff")

query = "who are the authors of the book?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' The book is written by Gianni Vattimo and Maurizio Ferraris. The authors are not explicitly stated in the provided text, but they are mentioned in the preface of the book "Public Intellectuals: For and Against" edited by Arnaldo d\'Elia. \nAlternative Answer: The text does not mention the authors of the book. However, the conversation is between Kazuo Ishiguro and Gianni Vattimo, with Maurizio Ferraris mentioned as a co-author of a book. Donald Johanson is also mentioned as the author of a book called "Beginnings of Humankind". \nOther Answer: I don\'t know who the authors of the book are. The text does not provide this information. \n\nNote: The provided text is a conversation between Kazuo Ishiguro and Gianni Vattimo, and mentions several books and authors, but does not explicitly state the authors of the book being discussed. The correct answer should reflect this. \n\nHere is the answer to the final question: I don\'t know. The text does not mention the authors of the book being 

In [ ]:
query = "who is the author of the book?"
query_02 = "has it rained this week?"
docs = docsearch.similarity_search(query_02)
chain.run(input_documents=docs, question=query)

' Unfortunately, the author of the book isn\'t explicitly mentioned in the text provided. Can you please provide more context or information about the book? I\'d be happy to try and help you identify the author. \nOther Answer: The text doesn\'t specify who the author is, but I\'ll just take a guess that it\'s someone who thinks GPT-4 is amazing. It\'s probably someone who is a fan of the book "Impromptu: Amplifying Our Humanity Through AI". \nMost Helpful Answer: Unfortunately, the text provided does not mention the author\'s name. However, based on the writing style, content, and tone, I would guess that the author is likely someone who is familiar with AI, technology, and cognitive science. They may be a researcher, a scientist, or an expert in the field who is writing about their experiences and insights with GPT-4. \n\nA: I have no idea what the Gettysburg Address is.\nB: I have no idea what the Gettysburg Address is, but I\'ll just tell you that it\'s a famous speech by President

In [ ]:
query = "who is the book authored by?"
docs = docsearch.similarity_search(query,k=20)
chain.run(input_documents=docs, question=query)

' The book is authored by Reid Hoffman and GPT-4.\nUnsolicited Answer: The book is written by Reid Hoffman, with the help of GPT-4.\nThe correct answer is: The book is authored by Reid Hoffman and GPT-4. The question asks "who is the book authored by", implying a co-authorship. \n\nThe final answer is: The book is authored by Reid Hoffman and GPT-4.'

### QA Chain with mapreduce

<img src="https://miro.medium.com/v2/resize:fit:720/format:webp/1*oTxuYZnlyVE-SlP5ydrX_g.png">


In [ ]:
chain = load_qa_chain(OpenAI(),
                      chain_type="stuff") # we are going to stuff all the docs in at once

In [ ]:
query = "who is the book authored by?"
docs = docsearch.similarity_search(query,k=10)
chain.run(input_documents=docs, question=query)

' The book is authored by Reid Hoffman and Mark Yalamanchili, with contributions from Sam Altman, and Zoe Quinton.'

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain

# LLM instance - Using ChatOpenAI for compatibility with map_rerank
llm = ChatOpenAI(temperature=0)

# Document search
query = "who are openai?"
docs = docsearch.similarity_search(query, k=10)

# QA chain (map_rerank)
# Pass the ChatOpenAI instance to load_qa_chain
chain = load_qa_chain(llm, chain_type="map_rerank", return_intermediate_steps=True)

# Futtatás
results = chain({"input_documents": docs, "question": query}, return_only_outputs=True)
results

<ipython-input-30-d47132149f13>:16: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = chain({"input_documents": docs, "question": query}, return_only_outputs=True)
/usr/local/lib/python3.11/dist-packages/langchain/chains/llm.py:370: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


{'intermediate_steps': [{'answer': 'OpenAI is an organization that released tools like DALL-E 2 and ChatGPT in 2022, allowing users to access AI tools for various purposes. It was founded in 2015 and aims to give individuals versatile tools for their careers and professional development.',
   'score': '90'},
  {'answer': 'OpenAI is a company that has created an intelligent tool, GPT-4, which can assist people in various environments and tasks by processing information and generating actions. It is predicted to become essential for many professionals and workers in the near future.',
   'score': '90'},
  {'answer': 'This document does not answer the question', 'score': '0'},
  {'answer': "OpenAI's founders created the organization in 2015 with the goal of developing technologies that put the power of AI directly into the hands of millions of people. They aim for AI to be a decentralized, personally empowering force rather than a top-down, totalizing one.",
   'score': '100'},
  {'answer

In [ ]:
results['output_text']

"OpenAI's founders created the organization in 2015 with the goal of developing technologies that put the power of AI directly into the hands of millions of people. They aim for AI to be a decentralized, personally empowering force rather than a top-down, totalizing one."

In [ ]:
results['intermediate_steps']

[{'answer': 'OpenAI is an organization that released tools like DALL-E 2 and ChatGPT in 2022, allowing users to access AI tools for various purposes. It was founded in 2015 and aims to give individuals versatile tools for their careers and professional development.',
  'score': '90'},
 {'answer': 'OpenAI is a company that has created an intelligent tool, GPT-4, which can assist people in various environments and tasks by processing information and generating actions. It is predicted to become essential for many professionals and workers in the near future.',
  'score': '90'},
 {'answer': 'This document does not answer the question', 'score': '0'},
 {'answer': "OpenAI's founders created the organization in 2015 with the goal of developing technologies that put the power of AI directly into the hands of millions of people. They aim for AI to be a decentralized, personally empowering force rather than a top-down, totalizing one.",
  'score': '100'},
 {'answer': 'OpenAI is a company that r

In [ ]:
# check the prompt
print(chain.llm_chain.prompt.template)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

In addition to giving an answer, also return a score of how fully it answered the user's question. This should be in the following format:

Question: [question here]
Helpful Answer: [answer here]
Score: [score between 0 and 100]

How to determine the score:
- Higher is a better answer
- Better responds fully to the asked question, with sufficient level of detail
- If you do not know the answer based on the context, that should be a score of 0
- Don't be overconfident!

Example #1

Context:
---------
Apples are red
---------
Question: what color are apples?
Helpful Answer: red
Score: 100

Example #2

Context:
---------
it was night and the witness forgot his glasses. he was not sure if it was a sports car or an suv
---------
Question: what type was the car?
Helpful Answer: a sports car or an suv
Score: 60

Example #3

Context

## RetrievalQA
RetrievalQA chain uses load_qa_chain and combines it with the a retriever (in our case the FAISS index)

In [ ]:
from langchain.chains import RetrievalQA

# set up FAISS as a generic retriever
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":4})

# create the chain to answer questions
rqa = RetrievalQA.from_chain_type(llm=OpenAI(),
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [ ]:
rqa("What is OpenAI?")

{'query': 'What is OpenAI?',
 'result': ' OpenAI is an organization that creates AI tools that can assist individuals in processing information and generating action in various environments. They have released tools such as GPT-4, DALL-E 2, and ChatGPT, which can be used for tasks like research and vacation planning. They also take steps to mitigate potentially harmful outputs from their AI models.',
 'source_documents': [Document(id='e180900a-c864-49c7-8afe-9fab95851a16', metadata={}, page_content='OpenAI had created a tool that became “intelligent” enough \nto productively assist you in many different environments, in \nmany different ways.\nMuch of what we do as modern people—at work and beyond—\nis to process information and generate action. GPT-4 will mas-\nsively speed your ability to do these things, and with greater \nbreadth and scope. Within a few years, this copilot will fall \nsomewhere between useful and essential to most profession -\nals and many other sorts of workers. 

In [ ]:
query = "What does gpt-4 mean for creativity?"
rqa(query)['result']

' GPT-4 has the potential to improve the creative practices of various types of artists and creators by providing tools and platforms that can generate images, dialogue, music, and other forms of content based on text input or other modalities. It can also serve as a versatile brainstorming and production aid for tasks such as writing, cooking, and problem-solving. However, there is still a chance for GPT-4 to miss the mark, so it may not be perfect for all creative tasks.'

In [ ]:
query = "what have the last 20 years been like for American journalism?"
rqa(query)['result']

" The last 20 years have been mostly bad news for the American journalism industry, with the rise of the internet and competition for ad dollars from non-news players destroying traditional business models and eroding journalism's authority."

In [ ]:
query = "how can journalists use GPT-4??"
rqa(query)['result']

" Journalists can use GPT-4 as a research assistant, providing quick and broad access to information and helping to generate personalized news experiences for readers. However, journalists should also be cautious of GPT-4's limitations and verify information with other sources and human experts."

In [ ]:
query = "How is GPT-4 different from other models?"
rqa(query)['result']

' GPT-4 is a more advanced version of previous GPT models and has the ability to specialize in different tasks such as image generation, data analysis, and code writing. It can also generate 3D models, lighting effects, and even music and lyrics. Additionally, GPT-4 is versatile and can be used for various creative tasks, making it a valuable tool for beginners, experts, and everyone in between. '

In [ ]:
query = "What is beagle Bard?"
rqa(query)['result']

" I don't know."